In [1]:
#https://www.nature.com/articles/s41467-018-08090-0  Mode Frequencies/LD params

import numpy as np
import matplotlib.pyplot as plt
import scipy.special as special

def eval_rabi_freq(nStart, nDelta, eta):
    """
    Calculates Rabi Frequency for nStart -> nStart + nDelta

    Args:
        nStart (int): Initial phonon state
        nDelta (int): Change in phonon number
        eta (float): Lamb-Dicke Parameter

    Returns:
        float: Rabi frequency
    """
    nEnd = nStart + nDelta

    nSmall = min([nStart, nEnd])
    nBig = max([nStart, nEnd])
    factor2 = np.exp(-0.5 * eta**2) * eta**(np.absolute(nDelta))
    factor3 = np.sqrt(np.math.factorial(nSmall)/np.math.factorial(nBig))
    factor4 = special.assoc_laguerre(eta**2, nSmall, np.absolute(nDelta))

    return factor2 * factor3 * factor4

def generate_ld_matrix(ld_param):
    """
    Generate 3x3 matrix containing the Lamb-Dicke parameters for 3 ions for the 3 modes of motion along each axis.
    
    Rows label modes
    Columns label the ions
    
    E.g) matrix[1][2]
        Row = 1 -> OP1 Mode
        Column = 2 -> 3rd Ion
    """
    matrix = np.empty([3, 3])
    
    for i in range(3):
        if i == 0: # Center-of-Mass Motion
            for j in range(3):
                matrix[i][j] = ld_param
        elif i == 1: # OP1
            matrix[i][0] = (ld_param * np.sqrt(3))/np.sqrt(2)
            matrix[i][1] = 0
            matrix[i][2] = (ld_param * np.sqrt(3))/np.sqrt(2)
        else:   # OP2
            matrix[i][0] = (ld_param * np.sqrt(3))/np.sqrt(6)
            matrix[i][1] = -2 * (ld_param * np.sqrt(3))/np.sqrt(6)
            matrix[i][2] = (ld_param * np.sqrt(3))/np.sqrt(6)
    
    return matrix

#### Infidelity Calculations

For a MS interaction involving ground-state cooled motional modes, the infidelity is approximately given by:

$ F = \frac{4}{5} \sum_p (|\alpha_{i,p}|^2 + |\alpha_{j,p}|^2), \quad \alpha_{i, p} = \int^\tau_0 \Omega(t) \sin[\psi(t)] e^{i\omega_p t}, \quad \psi(t) = \int^t_0 \mu(t')dt' $

The index $i$ labels the ions, and $p$ the motional modes, with an associated mode frequency $\omega_p$. Here, $\tau$ is the gate time, and $\Omega(t)$ and $\mu(t')$ are the time-dependent Rabi frequency and detuning respectively.

In the most basic instance of the MS gate, both the Rabi frequency and detuning are constants in time. This simplifies the equation significantly:

$ F = \frac{4}{5} \sum_p (|\alpha_{i,p}|^2 + |\alpha_{j,p}|^2), \quad \alpha_{i, p} = \Omega \int^\tau_0 \sin(\mu t) e^{i\omega_p t} $

### Three-ion System Set-up

In [5]:
# --------------------- Experimental Parameters (Based on Measurements on 6 December, 2023) --------------------

# --------------------- Mode Frequencies ---------------------
f0 = 112.94884

fAx_IP = np.average([f0 - 112.545007, 113.349302 - f0])
fAx_OP1 = np.average([f0 - 112.247337, 113.646767 - f0])
fAx_OP2 = np.average([f0 - 111.975974, 113.918956 - f0])

fR1_IP = np.average([f0 - 111.855983, 114.038652 - f0])
fR1_OP1 = np.average([f0 - 111.933887, 113.960546 - f0])
fR1_OP2 = np.average([f0 - 112.059230, 113.835470 - f0])

fR2_IP = np.average([f0 - 111.573692, 114.321367 - f0])
fR2_OP1 = np.average([f0 - 111.636050, 114.258107 - f0])
fR2_OP2 = np.average([f0 - 111.730128, 114.163908 - f0])

f_IP = [fAx_IP, fR1_IP, fR2_IP]
f_OP1 = [fAx_OP1, fR1_OP1, fR2_OP1]
f_OP2 = [fAx_OP2, fAx_OP1, fAx_OP2]

# # --------------------- Rabi Frequencies (Carrier) ---------------------
ion1_rabi = (np.pi/ (3.9 * 10**(-6)))
ion2_rabi = (np.pi/ (3.0 * 10**(-6)))
ion3_rabi = (np.pi/ (3.15 * 10**(-6)))

# Carrier crosstalks (ct)
ct_1_on_2 = 0
ct_1_on_3 = 0

ct_2_on_1 = (np.pi/ (13.8 * 10**(-6)))
ct_2_on_3 = 0

ct_3_on_1 = (np.pi/ (38 * 10**(-6)))
ct_3_on_2 = (np.pi/ (8.93 * 10**(-6)))

# --------------------- Lamb-Dicke Parameteres --------------------
ax_LD = 0.26 # GUESSED (no data to infer from)

# To calculate LD param for radials (same value because same projection angle)
#   R1_OP1 has pi-time of 53.3 (Ion1), 54.1 (Ion3)
#   LD of OP1 is LD of IP * Sqrt(3/2)
#   Thus, Omega(R1_OP1) = Sqrt(3/2) * LD * Omega(Carrier)
#   Or, LD = Pi/tPi(R1_OP1) / Pi/(tPi(Carrier)) / (Sqrt(3/2))
r1_LD = (0.048 + 0.059)/2 # Based on Pi-time of R
r2_LD = r1_LD

ax_ld = generate_ld_matrix(ax_LD)
r1_ld = generate_ld_matrix(r1_LD)
r2_ld = generate_ld_matrix(r2_LD)